In [9]:
import os
import numpy as np
import pandas as pd

from keras.models import load_model
from keras.preprocessing import image
from keras.applications import Xception

In [10]:
test_folder = "./test/"
feature_size = 299

model_path = "./Xception_top.h5"
model = load_model(model_path)

middle_faeture_shape = (10,10,2048)
X_model = Xception(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
X_model.load_weights("./xception_weights_notop.h5")

In [32]:
def preprocessing(x,output_size = None,pre_model = None):
    if pre_model is None:
        return x
    elif output_size is None:
        raise Exception('should use output_size for pre_model')
    else:
        middle_faeture = pre_model.predict(x)
        res = np.reshape(middle_faeture,(1,output_size[0] * output_size[1] * output_size[2]))
        return res

In [34]:
res = []

for f in os.listdir(test_folder):
    img = image.load_img(test_folder+f,target_size=(feature_size,feature_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    x = preprocessing(x,middle_faeture_shape,X_model)
    pre = model.predict(x)
    label_num = np.argmax(pre[0])
    if label_num == 0:
        label = 'norm'
    else:
        label = 'defect'+str(label_num)
    res.append({
        'file':f,
        'label':label
    })

In [35]:
df = pd.DataFrame(res)

In [36]:
df.to_csv('submit.csv',header=False,index=False)